<a href="https://colab.research.google.com/github/alanbseo/FlickrWorkshopUFZ/blob/master/UFZ_3_PretrainedCNN_retraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this tutorial, we will build our own classification model based on custom input data. We are going to use the pre-trained InceptionResNetV2, thus relatively small resource can make reasonable image classification working. 

References:

https://github.com/fchollet/deep-learning-with-python-notebooks

https://gist.github.com/liudanking

https://www.learnopencv.com/keras-tutorial-fine-tuning-using-pre-trained-models/

https://github.com/jkjung-avt/keras-cats-dogs-tutorial/blob/master/train_inceptionresnetv2.py

https://forums.fast.ai/t/globalaveragepooling2d-use/8358

https://www.tensorflow.org/hub/tutorials/image_retraining
 



In [0]:
# First some preparation scripts 
# I'll skip this part today, but please let me know if you want to get to know about those 
 
import numpy as np
import os

import cv2
import numpy as np
import pandas as pd
import skimage.io # scikit image 

### The below is to avoid certificat error 
# source: https://stackoverflow.com/questions/27835619/urllib-and-ssl-certificate-verify-failed-error
import requests

requests.packages.urllib3.disable_warnings()

import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context



In [2]:
# Import Keras and image manipulating libraries 

from keras.applications import inception_resnet_v2
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

 
from keras.applications.imagenet_utils import decode_predictions
import matplotlib.pyplot as plt

import keras
 

import csv
import pathlib
import fnmatch




from keras.preprocessing import image
from keras.applications import vgg16
 


from keras import backend as k


# to decode predictions
from keras.applications.imagenet_utils import decode_predictions

 

Using TensorFlow backend.


We need a set of images to teach the network about the new classes you want to recognize. We need to prepare input data set. In this example we use public dataset, later you can upload your own data which is samely structured. 

source: https://www.tensorflow.org/hub/tutorials/image_retraining

here's a later section that explains how to prepare your own images, but to make it easy we've

We will download creative-commons licensed flower photos to use initiall and organise folder structure in the following. First, to get the set of flower photos, run these commands:


In [6]:
!curl -LO http://download.tensorflow.org/example_images/flower_photos.tgz ~/
!tar xzf flower_photos.tgz
!ls /content/flower_photos

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  218M  100  218M    0     0   244M      0 --:--:-- --:--:-- --:--:--  243M
curl: (3) <url> malformed
daisy  dandelion  LICENSE.txt  roses  sunflowers  tulips


We split the photos into training and validation folders. 


In [0]:
# Typical input image sizes to a Convolutional Neural Network trained on ImageNet are 224×224, 227×227, 256×256, and 299×299; however, you may see other dimensions as well.
# VGG16, VGG19, and ResNet all accept 224×224 input images while Inception V3 and Xception require 299×299 pixel inputs, as demonstrated by the following code block:
# Nasnet large 331x331. If you train your own classes (i.e. the final Fully Connected/Dense layer), you can modify the img size 

img_width, img_height = 331, 331
  
train_data_dir = "Workshop_flower_photos/training"
validation_data_dir = "Workshop_flower_photos/validation"

nb_train_samples = 104
nb_validation_samples = 0

batch_size = 32 #  Means the number of images used in one batch. If you have 320 images and your batch size is 32, you need 10 internal iterations go through the data set once (which is called `one epoch')
# It is set  proportional to the training sample size. There are discussions but generally if you can afford, bigger is better. It

epochs = 100 # An epoch means the whole input dataset has been used for training the network. There are some heuristics to determine the maximum epoch. Also there is a way to stop the training based on the performance (callled  `Early stopping').

num_classes = 12
 



## Fine-tune a neural network on a new set of classes

The task of fine-tuning a network is to tweak the parameters of an already trained network so that it adapts to the new task at hand. As explained here, the initial layers learn very general features and as we go higher up the network, the layers tend to learn patterns more specific to the task it is being trained on. Thus, for fine-tuning, we want to keep the initial layers intact ( or freeze them ) and retrain the later layers for our task.
Thus, fine-tuning avoids both the limitations discussed above.
The amount of data required for training is not much because of two reasons. First, we are not training the entire network. Second, the part that is being trained is not trained from scratch.
Since the parameters that need to be updated is less, the amount of time needed will also be less.



References:

https://github.com/fchollet/deep-learning-with-python-notebooks

https://gist.github.com/liudanking

https://www.learnopencv.com/keras-tutorial-fine-tuning-using-pre-trained-models/

https://github.com/jkjung-avt/keras-cats-dogstutorial/blob/master/train_inceptionresnetv2.py

https://forums.fast.ai/t/globalaveragepooling2d-use/8358


In [0]:
print(os.listdir("."))


['.config', 'sample_data']


In [0]:

print(os.listdir("."))


['Saxony_Flickr_Selection', '.git', 'GoogleColab_setup.ipynb', 'UFZ_demo_M_RCNN_ver1.ipynb']


In [0]:


##### build our classifier model based on pre-trained InceptionResNetV2:


# Load the base pre-trained model

# do not include the top fully-connected layer
# 1. we don't include the top (fully connected) layers of InceptionResNetV2

model = inception_resnet_v2.InceptionResNetV2(include_top=False, weights='imagenet',input_tensor=None, input_shape=(img_width, img_height, 3))
# Freeze the layers which you don't want to train. Here I am freezing the all layers.
# i.e. freeze all InceptionV3 layers
for layer in model.layers[:]:
    layer.trainable = False

# New dataset is small and similar to original dataset:
# There is a problem of over-fitting, if we try to train the entire network. Since the data is similar to the original data, we expect higher-level features in the ConvNet to be relevant to this dataset as well. Hence, the best idea might be to train a linear classifier on the CNN codes.
# So lets freeze all the layers and train only the classifier

# first: train only the top layers

# for layer in net_final.layers[:FREEZE_LAYERS]:
#     layer.trainable = False
# for layer in net_final.layers[FREEZE_LAYERS:]:
#     layer.trainable = True

Instructions for updating:
Colocations handled automatically by placer.
219062272/219055592 [==============================] - 2s 0us/step


In [0]:

x = model.output

# Now that we have set the trainable parameters of our base network, we would like to add a classifier on top of the convolutional base. We will simply add a fully connected layer followed by a softmax layer with num_classes outputs.

# Adding custom Layer
# x = Flatten()(x)
# add a global spatial average pooling layer
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)


# If the network is stuck at 50% accuracy, there’s no reason to do any dropout. Dropout is a regularization process to avoid overfitting. But your problem is underfitting.
# x = Dropout(0.5)(x) # 50% dropout

# A Dense (fully connected) layer which generates softmax class score for each class
predictions = Dense(num_classes, activation='softmax', name='softmax')(x)



# creating the final model
# this is the model we will train
model_final = Model(inputs = model.input, outputs = predictions)


#Now we will be training only the classifiers (FC layers)

# compile the model (should be done *after* setting layers to non-trainable)

#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

# model_final.compile(optimizer='rmsprop', loss='categorical_crossentropy')



In [0]:

## load previously trained weights
model_final.load_weights('TrainedWeights/InceptionResnetV2_Saxony_retrain_flickr_final_epoch100_acc0.99.h5')



# Compile the final model using an Adam optimizer, with a low learning rate (since we are 'fine-tuning')
# For classification, categorical_crossentropy is most often used. It measures the information between the predicted and the true class labels similarly with the mutual information. It is
model_final.compile(optimizer=Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

print(model_final.summary())



In [0]:

## load previously trained weights
# model_final.load_weights('TrainedWeights/InceptionResnetV2_Saxony_retrain_flickr_final_epoch100_acc0.99.h5')



# Compile the final model using an Adam optimizer, with a low learning rate (since we are 'fine-tuning')
# For classification, categorical_crossentropy is most often used. It measures the information between the predicted and the true class labels similarly with the mutual information. It is
model_final.compile(optimizer=Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

print(model_final.summary())


# Initiate the train and test generators with data Augumentation
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    fill_mode = "nearest",
    zoom_range = 0.3,
    width_shift_range = 0.3,
    height_shift_range=0.3,
    rotation_range=30)
test_datagen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    fill_mode = "nearest",
    zoom_range = 0.3,
    width_shift_range = 0.3,
    height_shift_range=0.3,
    rotation_range=30)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = "categorical")
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size = (img_height, img_width),
    class_mode = "categorical")



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 331, 331, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 165, 165, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 165, 165, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 165, 165, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

NameError: ignored

In [0]:


# show class indices
print('****************')
for cls, idx in train_generator.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))
print('****************')



# Save the model according to the conditions
checkpoint = ModelCheckpoint("TrainedWeights/InceptionResnetV2_Saxony_retrain.h5", monitor='acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

# Setup the early stopping criteria
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')



# Re-train the model
history = model_final.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples,
    epochs = epochs,
 #   validation_data = validation_generator,
 #   validation_steps = nb_validation_samples,
    callbacks = [checkpoint, early])

# at this point, the top layers are well trained.


****************


NameError: ignored